In [23]:
fixed_lines = []

with open("../data/full_dataset-release.csv", "r", encoding="utf-8") as f:
    lines = f.readlines()

i = 0
while i < len(lines):
    line = lines[i].strip()

    if line and line[0].isdigit() and i + 1 < len(lines):
        next_line = lines[i + 1].strip()

        if next_line.startswith(","):
            combined = line + next_line
            fixed_lines.append(combined)
            i += 2
            continue

    fixed_lines.append(line)
    i += 1

with open("../data/fixed_dataset.csv", "w", encoding="utf-8") as f:
    for line in fixed_lines:
        f.write(line + "\n")

In [32]:
import pandas as pd

df = pd.read_csv('../data/fixed_dataset.csv')

df.columns = [
    'id', 'text', 'ticker', 'date', 'price',
    'return_1d', 'return_2d', 'return_3d', 'return_7d',
    'volume', 'volatility_10d', 'volatility_30d',
    'lstm_sentiment', 'textblob_sentiment'
]

df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)
df = df.dropna(subset=['date'])

df = df.drop(columns=['id', 'volatility_10d', 'volatility_30d', 'lstm_sentiment', 'textblob_sentiment'])

/var/folders/n0/gj667hr12xq0lkb3bn4rjxkc0000gp/T/ipykernel_34925/1296824117.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/fixed_dataset.csv')


In [25]:
df.head()

,text,ticker,date,price,return_1d,return_2d,return_3d,return_7d,volume
0,RT @robertoglezcano: @amazon #Patents Show Fl...,Amazon,2017-01-31,823.48,0.008379,0.014924,0.014924,-0.001263,3137196.0
1,@FAME95FM1 Jamaicans make money with @Payoneer...,PayPal,2017-01-31,39.78,0.002011,0.012318,0.012318,0.054801,9100057.0
2,@CBSi Jamaicans make money with @Payoneer @Pay...,PayPal,2017-01-31,39.78,0.002011,0.012318,0.012318,0.054801,9100057.0
3,@Hitz92fm Jamaicans make money with @Payoneer ...,PayPal,2017-01-31,39.78,0.002011,0.012318,0.012318,0.054801,9100057.0
4,RT @loadsofvans: Retweet this post &amp; follo...,Amazon,2017-01-31,823.48,0.008379,0.014924,0.014924,-0.001263,3137196.0


In [26]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

chunk_size = 50000
num_chunks = 7

chunks = [df[i * chunk_size : (i + 1) * chunk_size] for i in range(num_chunks)]

for i, chunk in enumerate(chunks):
    chunk.to_csv(f"./data/df_chunk_{i+1}.csv", index=False)

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
from tqdm import tqdm

In [28]:
model = AutoModelForSequenceClassification.from_pretrained("../models/finbert-finetuned1")
tokenizer = AutoTokenizer.from_pretrained("../models/finbert-finetuned1")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

In [29]:
chunk_paths = [f"./data/df_chunk_{i}.csv" for i in range(1, 8)]

for i, path in enumerate(chunk_paths):
    df = pd.read_csv(path)
    texts = df['text'].tolist()

    batch_size = 64
    preds, probs = [], []

    for j in tqdm(range(0, len(texts), batch_size), desc=f"Classifying Chunk {i+1}"):
        batch_texts = texts[j:j + batch_size]
        enc = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=128)
        enc = {k: v.to(device) for k, v in enc.items()}

        with torch.no_grad():
            logits = model(**enc).logits
            batch_probs = softmax(logits, dim=1)
            preds.extend(batch_probs.argmax(dim=1).cpu().tolist())
            probs.extend(batch_probs.cpu().tolist())
            
    df['sentiment_label'] = preds
    df['positive'] = [p[0] for p in probs]
    df['neutral'] = [p[1] for p in probs]
    df['negative'] = [p[2] for p in probs]

    df.to_csv(f"./data/df_chunk_{i+1}_with_sentiment.csv", index=False)

Classifying Chunk 7: 100%|██████████| 782/782 [1:28:30<00:00,  6.79s/it]    


In [34]:
import glob

all_chunks = pd.concat(
    [pd.read_csv(f) for f in sorted(glob.glob("./data/df_chunk_*_with_sentiment.csv"))],
    ignore_index=True
)

In [36]:
grouped = all_chunks.groupby(['ticker', 'date']).agg({
    'text': lambda x: list(x),
    'sentiment_label': lambda x: list(x),
    'positive': lambda x: list(x),
    'neutral': lambda x: list(x),
    'negative': lambda x: list(x)
}).reset_index()

In [37]:
grouped.to_csv("../data/full_sentiment_dataset.csv", index=False)